In [4]:
%matplotlib inline

# Packages
import os, glob, scipy, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Project directory
base_dir = os.path.realpath('..')
print(base_dir)

# Project-specific functions
funDir = os.path.join(base_dir,'Code/Functions')
print(funDir)
sys.path.append(funDir)
import choiceModels, costFunctions, penalizedModelFit, simulateModel

# General-use python functions
dbPath = '/'.join(base_dir.split('/')[0:4])
sys.path.append('%s/Python'%dbPath)
import FigureTools

/Users/jeroen/Dropbox (Brown)/PhD/0. Working folder/HMTG_followUp_final/ShareDataCode
/Users/jeroen/Dropbox (Brown)/PhD/0. Working folder/HMTG_followUp_final/ShareDataCode/Code/Functions


## Load Data

In [5]:
study1 = pd.read_csv('%s/Data/Study1/HMTG/allDataLong.csv'%baseDir, header = None)
study1.columns = ['sub','block','trial','inv','baseMult','mult','exp','ret']
exclude = pd.read_csv('%s/Data/Study1/HMTG/exclude.csv'%baseDir, header = None).values.T[0]
print(len(study1))
study1 = study1.loc[~study1['sub'].isin(exclude),:]
study1.head()
print(len(study1))
print(len(study1['sub'].unique()))

16640
16320
102


## Fit models

Here is some example code using the models and cost functions defined in the Functions folder:

In [33]:
niter = 3
baseMult = 4
model = costFunctions.MP_costfun_ppSOE
modelName = 'MP_ppSOE'
best_fits = pd.DataFrame(columns = ['sub','baseMult','model','theta','phi','SSE'])
for sub in [1,2]:
    # Set up fitting output dataframe:
    sub_fits = pd.DataFrame(columns = ['sub','baseMult','model','theta','phi','SSE'])
    # Select subject data, keep only trials with nonzero investment:
    subDat = study1.query('sub == @sub and baseMult == @baseMult and inv > 0').copy().reset_index(drop=True)
    # Fit model between parameters on domains [0,.5] (theta) and [-.1,.1] (phi)
    bounds_lower = [0,-.1]
    bounds_upper = [.5,.1]
    for i in range(niter):
        randos = np.random.rand(2)
        x0 = np.array(bounds_lower) + np.multiply(randos, np.array(bounds_upper) - np.array(bounds_lower))
        out = least_squares(model, x0, bounds = [bounds_lower, bounds_upper],
                 args = ([subDat]), diff_step = .005)
        print(x0, out.x, out.cost)
        sub_fits = sub_fits.append(pd.DataFrame([[sub,baseMult,modelName,out.x[0],out.x[1],out.cost]],
                                               columns = sub_fits.columns))
    # Select best-fitting parameters for participant and add to results dataframe:
    sub_fits = sub_fits.sort_values(by = 'SSE', ascending = True)
    best_fits = best_fits.append(sub_fits.iloc[0,:])

[0.06412323 0.01511354] [ 0.08738989 -0.00213702] 84.0
[0.28125205 0.05112751] [0.19277217 0.05112751] 238.5
[0.47922263 0.0702712 ] [0.17894786 0.0702712 ] 195.0
[ 0.03932602 -0.01090841] [0.07953587 0.00113294] 3169.5
[ 0.03473394 -0.01667751] [0.15182059 0.00117889] 2549.0
[0.37100653 0.0048783 ] [0.48903136 0.02674215] 3.5


In [34]:
best_fits

,sub,baseMult,model,theta,phi,SSE
0,1,4,MP_ppSOE,0.087390,-0.002137,84.0
0,2,4,MP_ppSOE,0.489031,0.026742,3.5


These models tend to get stuck in local minima, depending on the fitting algorithm used and settings like diff_step. To maximize the likelihood that our fitted model parameters were global maxima within the parameter bounds, we ran this fitting procedure 1000 times per participant on a computing cluster.